# OPC risk analaysis

in the few following paragraph we will take a look at some of your electricity meters, and analyze their prediction errors alongside some other markers. the code that was used to create these plots and analysis is hidden but is still available if you like.

In [ ]:
#@title
# importing the required dependencies

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.express as px

In [ ]:
#@title
from google.colab import drive
drive.mount('/content/drive')
by_error_path = "/content/drive/MyDrive/by_error.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title
# lets take a look at the stations we have

by_error = pd.read_csv(by_error_path,parse_dates=["From"])


# Data

First lets take a look at our data, all the measurements are composed of the entity, site, meter, the time it was taken in, the electricity consumption, the forecast for that consumption, and the calculated total error.

we included only the observation with positive errors, as for now overestimations interest us less than underestimations.

In [ ]:
#@title
# only the positive errors (underestimations) interest us right now
by_error_pos = by_error[by_error['total error'] > 0]
by_error_pos = by_error_pos.loc[(by_error_pos.From != "2021-10-31 01:30:00")]

by_error_pos.head()

,Unnamed: 0,Entity,Site,Meter,From,total error,Actual kWh,Forecast kWh
8,8,Rotem,ORL,ORL,2021-10-10 03:30:00,8395.663,30212.00,21816.337
9,9,Rotem,ORL,ORL,2021-10-10 04:30:00,8304.356,30179.52,21875.164
10,10,Rotem,ORL,ORL,2021-10-10 06:00:00,8285.503,30750.72,22465.217
11,11,Rotem,ORL,ORL,2021-10-10 04:00:00,8170.345,30131.36,21961.015
12,12,Rotem,ORL,ORL,2021-10-10 15:30:00,7968.785,30391.20,22422.415


Let's take a quick look at the relative part of each station in the aggregate error, and aggregate consumption, so we can get a good idea about the importance of our different stations:

In [ ]:
#@title
# extracting the total consumption and errors
consumption = by_error_pos.groupby('Site')['Actual kWh'].sum().sort_values(ascending=False)
max_error = by_error_pos.groupby('Site')['total error'].sum().sort_values(ascending=False)


cuttof = 20
rest_cons = pd.Series({'rest of stations':consumption.iloc[cuttof:].sum()})
consumption_short = consumption.iloc[:cuttof].append(rest_cons)

rest_error = pd.Series({'rest of stations':max_error.iloc[cuttof:].sum()})
error_short = max_error.iloc[:cuttof].append(rest)


In [ ]:
#@title
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])


fig.add_trace(go.Pie(
     values=error_short,
     labels=error_short.index,
     name="total error"), 
     row=1, col=1)

fig.add_trace(go.Pie(
     values=consumption_short,
     labels=consumption_short.index,
     name="total consumption"), 
    row=1, col=2)

fig.show()

As we can see, there is a match between the highest consuming stations, and the stations in which the maximum errors are the highest.

Notice that the first 5 individual meters in both pie charts are:

1. Intel F28
2. CAOL
3. ORL
4. Hadera paper main
5. Intel F28A 1


# Max error vs Total consumption

We should take a look at the meters which had the higest errors,
if we plot the maximum error against the total consumption, we can get an idea
for how much a risk a site poses relative to its importance

In [ ]:
#@title
# extracting the total consumption asnd errors
consumption = by_error_pos.groupby('Site')['Actual kWh'].sum()
max_error = by_error_pos.groupby('Site')['total error'].max()

# renaming for convenience
error_vs_cons = pd.DataFrame([consumption,max_error]).T.rename(columns={'Actual kWh':'total consumption','total error':'max error'}) 
error_vs_cons['Site'] = error_vs_cons.index


# annotating only the highest ones
error_vs_cons.loc[(error_vs_cons['max error']<900),'Site'] = ''


In [ ]:
#@title

fig = px.scatter(error_vs_cons, x="total consumption", y="max error",text='Site')
fig.update_traces(textposition='top center',textfont_size=10)
fig.show()

And we do still see that the riskiest meters are also the ones who consume the most. theres almost a linear relationship between the total consumption and the maximum error variables

# Error count per meter

It's also important to consider the **amount** of errors on a certain meters.
lets take the worst 5000 errors we had, and see if any particular meters caused
most of them:

In [ ]:
#@title
# lets take the worst 5000 errors
error_counts = by_error[:5000].groupby('Site').count()['From']

fig = px.bar(error_counts, y=error_counts, x=error_counts.index)
fig.show()

We can see that most of our errors are being made by about 10 Meters

# Riskiest stations distribution

we can get some insights about their distribution and characterize our riskiest 10 meters 

In [ ]:
#@title

prob_sites = by_error_pos[:5000]['Site'].unique()
prob_obs = by_error_pos.loc[(by_error['Site'].isin(prob_sites))]

# pivoting the total error by time and site
pt = pd.pivot_table(prob_obs,values = "total error", index = 'From', columns = 'Site')

In [ ]:
#@title

fig = go.Figure()
for col in pt.columns:
  fig.add_trace(go.Box(x=pt[col],name=col))
fig.show()